In [1]:
pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
api_key = os.getenv("DEEPSEEK_API_KEY")

'sk-ad03f2e90cfe4856a08220c42ecad2e3'

In [4]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [6]:
len(text_lines)

72

In [21]:
from openai import OpenAI
deepseek_client = OpenAI(
    api_key = api_key,
    base_url="https://api.deepseek.com/v1", 
)

In [12]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836056  0.07163018 -0.01130064 -0.03789344 -0.03320646 -0.01318444
 -0.03041711 -0.02269505 -0.02317867 -0.00426023]


In [13]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding[:10])

[-0.04836056  0.07163018 -0.01130064 -0.03789344 -0.03320646 -0.01318444
 -0.03041711 -0.02269505 -0.02317867 -0.00426023]


In [14]:
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_demo.db")
collection_name = "my_rag_collection"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name = collection_name,
    dimension = embedding_dim,
    metric_type = "IP",
    consistency_level = "Strong",
)

from tqdm import tqdm
data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
for i,line in enumerate(tqdm(text_lines,desc="Creating embeddings")):
    data.append({"id":i,"vector":doc_embeddings[i],"text":line})

milvus_client.insert(collection_name=collection_name,data=data)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Creating embeddings: 100%|██████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 1027176.49it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

In [16]:
question = "How is data stored in milvus?"

search_res = milvus_client.search(
    collection_name = collection_name,
    data = embedding_model.encode_queries([question]),
    limit = 3,
    search_params = {"metric_type":"IP","params":{}},
    output_fields = ["text"],
)

import json
retrieved_lines_with_distances = [
    (res["entity"]["text"],res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances,indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.6572662591934204
    ],
    [
        "How does Milvus flush data?\n\nMilvus returns success when inserted data are loaded to t

In [17]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
print(context)

 Where does Milvus store data?

Milvus deals with two types of data, inserted data and metadata. 

Inserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).

Metadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.

###
How does Milvus flush data?

Milvus returns success when inserted data are loaded to the message queue. However, the data are not yet flushed to the disk. Then Milv

In [19]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""
print(USER_PROMPT)


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
 Where does Milvus store data?

Milvus deals with two types of data, inserted data and metadata. 

Inserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).

Metadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.

###
How does Milvus flush data?

Milvus returns success

In [22]:
response = deepseek_client.chat.completions.create(
    model = "deepseek-chat",
    messages = [
        {"role":"system","content":SYSTEM_PROMPT},
        {"role":"user","content":USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

In Milvus, data is stored in two forms: inserted data and metadata. Inserted data, which includes vector data, scalar data, and collection-specific schema, is stored as incremental logs in persistent storage such as MinIO, AWS S3, Google Cloud Storage, Azure Blob Storage, Alibaba Cloud OSS, or Tencent Cloud Object Storage. Metadata, generated within Milvus, is stored in etcd.

<translated>
在 Milvus 中，数据以两种形式存储：插入的数据和元数据。插入的数据，包括向量数据、标量数据和集合特定模式，以增量日志的形式存储在持久存储中，如 MinIO、AWS S3、Google Cloud Storage、Azure Blob Storage、阿里云 OSS 或腾讯云对象存储。元数据在 Milvus 内部生成，存储在 etcd 中。
</translated>
